In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge 

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td:td.total_seconds() / 60)

In [3]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [4]:
((df.duration >=1) & (df.duration <=60)).mean()

0.9812202822125979

In [5]:
categorical =[ 'PULocationID','DOLocationID']
 
dv = DictVectorizer()

train_dicts = df[categorical].to_dict(orient='records')
X_train =dv.fit_transform(train_dicts)
X_train

<3066766x2 sparse matrix of type '<class 'numpy.float64'>'
	with 6133532 stored elements in Compressed Sparse Row format>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td:td.total_seconds() / 60)
    
    df =df[((df.duration >=1) & (df.duration <=60))]
    
    categorical =['PULocationID','DOLocationID']
    #numerical =['trip_distance']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [8]:
len(df_train),len(df_val)

(3009173, 2855951)

In [9]:
#df_train['PU_DO']=df_train['PULocationID'] + '_' + df_train['DOLocationID']
#df_val['PU_DO']=df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
#categorical =['PU_DO']  
categorical =['PULocationID','DOLocationID']
#numerical =['trip_distance']

dv = DictVectorizer()
 
train_dicts = df_train[categorical].to_dict(orient='records') #+  numerical
X_train =dv.fit_transform(train_dicts)


val_dicts = df_val[categorical].to_dict(orient='records') #+  numerical 
X_val = dv.transform(val_dicts)

In [11]:
target ='duration'
y_train= df_train[target].values
y_val= df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train,y_pred,squared=False)

7.6492610279057605

In [14]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

7.81183265470218